<a href="https://colab.research.google.com/github/hsabaghpour/GraphBased_LTE_HandoverOptimization/blob/main/Handover_Optimization_using_Graph_Neural_Networks_(GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem Overview:

	•	Objective: Optimize the handover success rate in an LTE network by using a Graph Neural Network (GNN) to predict and improve NBR (neighbor relations) between eNodeBs (base stations).
	•	Nodes: eNodeBs represent the base stations in the LTE network.
	•	Edges: NBR relations (handover success rates) between eNodeBs, weighted by success rate values.

We will use PyTorch Geometric, a powerful library that extends PyTorch to work with graphs and GNNs.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx

	•	torch: Standard PyTorch library for defining the GNN and training the model.
	•	PyTorch Geometric (torch_geometric): A library for deep learning on graphs, which provides GNN layers like GCNConv for graph convolutions.
	•	NetworkX (nx): Used to create the LTE graph with eNodeBs and NBR relations.

In [ ]:
graph_data = {
    ('eNodeB1', 'eNodeB2'): 0.9,
    ('eNodeB1', 'eNodeB3'): 0.85,
    ('eNodeB2', 'eNodeB3'): 0.6,
    ('eNodeB3', 'eNodeB4'): 0.7,
    ('eNodeB1', 'eNodeB4'): 0.4,
}

	•	The dictionary contains pairs of eNodeBs as edges and their associated handover success rates as weights.
	•	Example: eNodeB1 and eNodeB2 have a handover success rate of 0.9.

	•	We initialize a NetworkX graph G and add edges representing NBR relations between eNodeBs.
	•	The weight on each edge is the handover success rate.

In [ ]:
G = nx.Graph()
for (node1, node2), success_rate in graph_data.items():
    G.add_edge(node1, node2, weight=success_rate)

#Convert to PyTorch Geometric Graph Format
	•	from_networkx(): Converts the NetworkX graph into a PyTorch Geometric data object that can be used with GNN layers.
	•	Node Features (data.x): Dummy node features are provided as an example. In a real scenario, node features could represent the characteristics of each eNodeB (e.g., traffic load, signal strength).
	•	Edge Weights (data.edge_attr): The success rates (weights of edges) are used as features for the edges.

In [ ]:
data = from_networkx(G)
data.x = torch.tensor([[0], [1], [1], [1], [0]], dtype=torch.float)  # Example node features
data.edge_attr = torch.tensor([list(graph_data.values())], dtype=torch.float).T  # Edge weights as features

**Define the GCN (Graph Convolutional Network) Model**

	•	The GCN (Graph Convolutional Network) consists of two convolutional layers (GCNConv):
	•	conv1: Takes the node features as input and produces a 16-dimensional feature vector for each node.
	•	conv2: Further transforms the 16-dimensional features into 32 dimensions.
	•	ReLU Activation: Introduces non-linearity to the model.
	•	Log Softmax: The output is a log probability distribution for each node (or edge).

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(1, 16)  # First GCN layer (input feature size is 1)
        self.conv2 = GCNConv(16, 32)  # Second GCN layer (output feature size is 32)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

#**Model Training Setup**

	•	Model: The GCN model is instantiated.
	•	Optimizer: We use the Adam optimizer with a learning rate of 0.01.
	•	Loss Function: CrossEntropyLoss is used to compare the model’s predictions against the true labels.

7. Train the Model

In [ ]:
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

	•	Training Loop: The model is trained for 100 epochs. In each epoch:
	•	The model performs a forward pass on the graph data.
	•	The loss is computed between the model’s predictions and the true labels ([0, 1, 1, 1, 0]), representing whether the handover was successful (1) or failed (0).
	•	The model’s parameters are updated using backpropagation and the optimizer.

In [ ]:
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)  # Forward pass
    loss = criterion(out, torch.tensor([0, 1, 1, 1, 0]))  # Example target
    loss.backward()  # Backward pass
    optimizer.step()  # Update model parameters

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# ** Model Prediction**


	•	After training, the model is set to **evaluation mode.**
	•	The trained model is used to predict the success of NBR relations based on the learned representations of nodes and edges.
	•	The output is a log probability distribution over the classes (success/failure of handovers).

Explanation of GNN Approach:

	1.	Graph Construction: The LTE network is represented as a graph, where nodes are eNodeBs, and edges represent NBR relations with handover success rates as weights.
	2.	Graph Representation Learning: The GCN model learns node and edge representations through graph convolutions, using node features and edge weights to propagate information across the graph. This helps the model capture the structure of the network and the relationships between eNodeBs.
	3.	Handover Prediction: After training, the model can predict the success or failure of NBR relations. Successful relations (high handover success rates) are kept, while unsuccessful relations can be removed or adjusted to optimize the network.

Key Concepts:

	•	Graph Neural Network (GNN): Learns to optimize NBR relations by leveraging both node (eNodeB) features and edge (handover success rate) features.
	•	GCNConv Layers: These layers aggregate and transform node features by considering the structure of the graph, which allows the model to learn from the relationships between nodes.
	•	Edge Weights (Handover Success Rates): These weights influence the training process by serving as important features for learning how successful or unsuccessful handovers occur.

Outcome:

The GNN model provides predictions on which NBR relations are likely to be successful and which may fail. This enables the optimization of the handover process by focusing on strengthening successful relations and removing or adjusting weaker ones.